In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [4]:
# reading transaction table
transaction = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='Transactions', header=1)
transaction.sample()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
16111,16112,26,1150,2017-02-03,1.0,Approved,WeareA2B,Standard,medium,medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# checking for shape of the table
transaction.shape

(20000, 26)

In [6]:
# dtype of all the columns
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

In [7]:
# dropping 'Unnamed' columns
unnanmed_columns = [column for column in transaction.columns if 'Unnamed' in column]
transaction.drop(columns=unnanmed_columns, inplace=True)
transaction.columns

Index(['transaction_id', 'product_id', 'customer_id', 'transaction_date',
       'online_order', 'order_status', 'brand', 'product_line',
       'product_class', 'product_size', 'list_price', 'standard_cost',
       'product_first_sold_date'],
      dtype='object')

In [8]:
transaction.sample(5)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
825,826,2,1307,2017-09-20,0.0,Approved,Giant Bicycles,Road,low,small,590.26,525.33,36668.0
16193,16194,58,178,2017-10-27,0.0,Approved,OHM Cycles,Standard,medium,medium,912.52,141.40,42295.0
17340,17341,6,1670,2017-03-02,0.0,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,37659.0
16924,16925,32,499,2017-01-16,0.0,Approved,Giant Bicycles,Standard,medium,medium,642.70,211.37,37337.0
11015,11016,0,2456,2017-05-12,NaN,Approved,NaN,NaN,NaN,NaN,1719.95,NaN,NaN


## Data Preprocessing

### Null values and Incorrect values

In [9]:
# checking for Null values and percentage of Null values
for column in transaction.columns:
    if transaction[column].isna().sum() > 0:
        print({column: transaction[column].isna().sum() * 100 / len(transaction)})

{'online_order': 1.8}
{'brand': 0.985}
{'product_line': 0.985}
{'product_class': 0.985}
{'product_size': 0.985}
{'standard_cost': 0.985}
{'product_first_sold_date': 0.985}


In [10]:
transaction[transaction.brand.isna()].product_id.unique()

array([0], dtype=int64)

*No feature has high level of Null values. However for Product ID: 0, we don't have any details.*

In [11]:
# getting list of all the Numerical variables with Null values
numerical_features_na = [column for column in transaction.columns if transaction[column].dtype != 'O'
                        and transaction[column].isna().sum() > 0]
numerical_features_na

['online_order', 'standard_cost', 'product_first_sold_date']

In [12]:
# replacing Null values for Numerical variables
transaction['online_order'].fillna(transaction['online_order'].mode()[0], inplace=True)
transaction['standard_cost'].fillna(transaction['standard_cost'].median(), inplace=True)
transaction['product_first_sold_date'].fillna(transaction['product_first_sold_date'].mode()[0], inplace=True)

In [13]:
# checking for unique values of all the categorical features
categorical_features = [column for column in transaction.columns if transaction[column].dtype == 'O']
for feature in categorical_features:
    print(feature)
    print(transaction[feature].unique(),'\n')

order_status
['Approved' 'Cancelled'] 

brand
['Solex' 'Trek Bicycles' 'OHM Cycles' 'Norco Bicycles' 'Giant Bicycles'
 'WeareA2B' nan] 

product_line
['Standard' 'Road' 'Mountain' 'Touring' nan] 

product_class
['medium' 'low' 'high' nan] 

product_size
['medium' 'large' 'small' nan] 



In [14]:
# checking for categorical variables with Null values
categorical_features_na = [column for column in transaction.columns if transaction[column].dtype == 'O'
                       and transaction[column].isna().sum() > 0]
categorical_features_na

['brand', 'product_line', 'product_class', 'product_size']

In [15]:
# handling Null values for categorical features
for feature in categorical_features_na:
    transaction[feature].fillna('Missing', inplace=True)

In [16]:
# rechecking for Null values
transaction.isna().sum()

transaction_id             0
product_id                 0
customer_id                0
transaction_date           0
online_order               0
order_status               0
brand                      0
product_line               0
product_class              0
product_size               0
list_price                 0
standard_cost              0
product_first_sold_date    0
dtype: int64

In [17]:
transaction.sample(5)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
3547,3548,35,3448,2017-11-11,1.0,Approved,Giant Bicycles,Standard,medium,medium,1403.50,954.82,42688.0
12419,12420,88,1395,2017-02-25,0.0,Approved,Norco Bicycles,Standard,high,small,1661.92,1479.11,34244.0
18873,18874,37,626,2017-09-15,1.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
12051,12052,10,2424,2017-09-02,0.0,Approved,WeareA2B,Touring,medium,medium,1466.68,363.25,41701.0
14330,14331,69,1592,2017-06-26,0.0,Approved,Giant Bicycles,Road,medium,medium,792.90,594.68,42710.0


In [18]:
# adding a profit column
transaction['Profit'] = transaction.list_price - transaction.standard_cost

In [19]:
transaction.sample(5)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,Profit
5196,5197,87,2697,2017-09-21,1.0,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71,40410.0,1592.19
10137,10138,71,3202,2017-12-19,0.0,Approved,Solex,Standard,high,large,1842.92,1105.75,40553.0,737.17
13947,13948,59,1799,2017-11-01,1.0,Approved,Solex,Standard,medium,large,1061.56,733.58,34170.0,327.98
17563,17564,22,1451,2017-03-10,1.0,Approved,WeareA2B,Standard,medium,medium,60.34,45.26,40670.0,15.08
3272,3273,0,1731,2017-10-25,1.0,Approved,Trek Bicycles,Standard,medium,medium,499.53,388.72,33552.0,110.81


In [20]:
# dropping first sold date as it doesn't make any sense
transaction.drop(columns='product_first_sold_date', inplace=True)

In [22]:
# saving as excel
transaction.to_excel(r'C:\Users\Mayank\Downloads\Transaction.xlsx', index=False)